In [18]:
## Combining Dataset and Keeping Top Customers with Optimal Revenue
import pandas as pd
import os

os.chdir('../ModelOutput')

# Step 1: Load the files
cc_pred = pd.read_csv('CC_predictions.csv')  # Credit Card predictions
cl_pred = pd.read_csv('CL_predictions.csv')  # Consumer Loan predictions
mf_pred = pd.read_csv('MF_predictions.csv')  # Mutual Fund predictions

# Step 2: Perform left joins on the 'Client' column
combined_df = cc_pred.merge(cl_pred, on='Client', how='left')  # Left join CC and CL
combined_df = combined_df.merge(mf_pred, on='Client', how='left')  # Left join with MF

# Step 3: Display the combined DataFrame
print("Combined DataFrame:")
print(combined_df.head())


Combined DataFrame:
   Client  Sale_CC  Predicted_Probability_Sale_CC  Predicted_Sale_CC  \
0       1      0.0                       0.600879                1.0   
1       2      0.0                       0.652222                1.0   
2       3      NaN                       0.507355                1.0   
3       4      NaN                       0.550932                1.0   
4       5      NaN                       0.668801                1.0   

   Revenue_CC  Sale_CL  Predicted_Probability_Sale_CL  Predicted_Sale_CL  \
0    0.000000      0.0                       0.156128                0.0   
1    0.000000      0.0                       0.095612                0.0   
2    4.376964      NaN                       0.562485                1.0   
3    4.376964      NaN                       0.394612                0.0   
4    4.376964      NaN                       0.049666                0.0   

   Revenue_CL  Sale_MF  Predicted_Probability_Sale_MF  Predicted_Sale_MF  \
0    0.000000 

## Calculating Expected Revenue for Each Product and each customer

In [19]:
combined_df['ExpectedRev_MF'] = combined_df['Predicted_Probability_Sale_MF']* combined_df['Predicted_Probability_Sale_MF']*combined_df['Revenue_MF']

combined_df['ExpectedRev_CL'] = combined_df['Predicted_Probability_Sale_CL']* combined_df['Predicted_Probability_Sale_CL']*combined_df['Revenue_CL']

combined_df['ExpectedRev_CC'] = combined_df['Predicted_Probability_Sale_CC']* combined_df['Predicted_Probability_Sale_CC']*combined_df['Revenue_CC']

## Find the Score which is the Maximum Expected Revenue for each client

In [20]:
combined_df['Score'] = combined_df[['ExpectedRev_MF', 'ExpectedRev_CL', 'ExpectedRev_CC']].max(axis=1)

## Find the Product with the Maximum Expected Revenue for each client

In [21]:
combined_df['Top_Offer'] = combined_df[['ExpectedRev_MF', 'ExpectedRev_CL', 'ExpectedRev_CC']].idxmax(axis=1)
combined_df['Top_Offer'] = combined_df['Top_Offer'].str.replace('ExpectedRev_', '')  # Clean the column names

/var/folders/9n/q66b3sm128dc1lgchwd8mb280000gn/T/ipykernel_5387/4037252610.py:1: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  combined_df['Top_Offer'] = combined_df[['ExpectedRev_MF', 'ExpectedRev_CL', 'ExpectedRev_CC']].idxmax(axis=1)


## Find the top 100 customers to target

In [22]:
top_100_customers = combined_df.nlargest(100, 'Score')
top_100_clients_summary = top_100_customers[['Client', 'Score', 'Top_Offer']]
print(top_100_clients_summary)

write_path = 'Top_100_Customers.csv'
top_100_clients_summary.to_csv(write_path, index=False)

      Client       Score Top_Offer
1334    1335  147.296983        CC
965      966   89.519939        CC
447      448   65.772949        CC
26        27   56.110655        MF
1292    1293   47.967569        CC
...      ...         ...       ...
44        45    4.815518        CC
582      583    4.808965        CL
1072    1073    4.764471        CL
0          1    4.757008        MF
173      174    4.753162        CL

[100 rows x 3 columns]


## Expected Revenue from Top 100 customers

In [23]:
Total_expected_revenue = top_100_customers['Score'].sum()
print(f"Total Expected Revenue from Top 100 Customers: {Total_expected_revenue}")

Total Expected Revenue from Top 100 Customers: 1337.3519696426185
